second version of training that uses aggregation mlp

In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
class InductiveLPDataset(DisjointInductivePathDataset):
    """An inductive link prediction dataset for the ILPC 2022 Challenge."""

    
    
    
    def __init__(self , **kwargs):
        """Initialize the inductive link prediction dataset.

        :param size: "small" or "large"
        :param kwargs: keyword arguments to forward to the base dataset class, cf. DisjointInductivePathDataset
        """
        DATA_TYPE = "_fully_inductive_io.tsv"
        TRAIN_PATH = "MSCallGraph_train" + DATA_TYPE
        TEST_PATH = "MSCallGraph_test" + DATA_TYPE
        VALIDATE_PATH = "MSCallGraph_validation" + DATA_TYPE
        INFERENCE_PATH = "MSCallGraph_inference" + DATA_TYPE


        super().__init__(
            transductive_training_path=os.getcwd()+"/"+TRAIN_PATH,
            inductive_inference_path=os.getcwd()+"/"+INFERENCE_PATH,
            inductive_validation_path=os.getcwd()+"/"+VALIDATE_PATH,
            inductive_testing_path=os.getcwd()+"/"+TEST_PATH,
            create_inverse_triples=True,
            eager=True,
            **kwargs
        )


In [3]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [4]:
dataset = InductiveLPDataset()

In [5]:
model_name = 'nodepiece_inductive_2_io'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
num_tokens = 20
embedding_dim = 200

In [8]:
model = InductiveNodePiece(
        triples_factory=dataset.transductive_training,
        inference_factory=dataset.inductive_inference,
        random_seed = seed,
        loss = loss,
        num_tokens = num_tokens,
        embedding_dim = embedding_dim,
        aggregation = "mlp"
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

sampling:   0%|          | 0.00/8.74k [00:00<?, ?it/s]

No symbolic computation of output shape.


sampling:   0%|          | 0.00/3.59k [00:00<?, ?it/s]

No symbolic computation of output shape.


Number of parameters: 1684000
Space occupied: 6736000 bytes


In [9]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory nodepiece_inductive_2_io created successfully!


In [10]:
learning_rate = 1e-3
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [11]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        mode=TRAINING,
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        mode=VALIDATION,
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        mode=TESTING,
        metrics = metrics,
        add_defaults=False
    )

In [12]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.inductive_inference,
    evaluation_triples_factory = dataset.inductive_validation,
    result_tracker = tracker

)



In [13]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.transductive_training,
        model=model,
        mode=TRAINING,  # must be specified for the inductive setup
        result_tracker=tracker,
        optimizer=optimizer
)

In [14]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.transductive_training,
        num_epochs=num_epochs,
        stopper = stopper
        
    )
training_duration = time.time() - training_start

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 48.09002685546875
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.003253736564792886
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.012060664337184888
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.007657200450988887
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0024314632173627615
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.012044288218021393
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.007237876299768686
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.002146632623998567
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.012029908760796475
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.007088270692397523
Metric: validation.head.optimistic.hits_at_1 = 0.0005803830528148578
Metric: validation.tail.optimistic.hits_at_1 = 0.0046430644225188625
Metric: validation.both.optimistic.hits_at_1 = 

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 2
Metric: loss = 10.891788005828857
Step: 2
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.00781750330597566
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.01641915798677506
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.01211833064637536
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.007635434158146381
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.016412949189543724
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.012024191208183765
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.007548145021598107
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.016407875862633068
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.011978010442115588
Metric: validation.head.optimistic.hits_at_1 = 0.0002901915264074289
Metric: validation.tail.optimistic.hits_at_1 = 0.0020313406848520023
Metric: validation.both.optimistic.hits_at_1 = 0.0

...

Step: 90
Metric: loss = 0.42103105783462524
Step: 90
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.5358936264898435
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.05810383934711822
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.2969987329184809
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.025059424340724945
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.04558028280735016
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.03531985357403755
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.023489604203250213
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.04295532570904432
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.03322246495614727
Metric: validation.head.optimistic.hits_at_1 = 0.5220545560069646
Metric: validation.tail.optimistic.hits_at_1 = 0.031050493325594893
Metric: validation.both.optimistic.hits_at_1 = 0.27655252

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 91
Metric: loss = 0.4184076339006424
Step: 91
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.5316251773595632
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.053896678009498954
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.2927609276845311
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.026378611102700233
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.042299266904592514
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.03433893993496895
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.024893282180195816
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.03977069164008215
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.032331986910138985
Metric: validation.head.optimistic.hits_at_1 = 0.5153801508995938
Metric: validation.tail.optimistic.hits_at_1 = 0.02901915264074289
Metric: validation.both.optimistic.hits_at_1 = 0.2721996

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 92
Metric: loss = 0.41373687982559204
Step: 92
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.5343839608367512
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.058041661144165056
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.2962128109904581
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.02629224769771099
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.045524850487709045
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.03590855002403259
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.024760872491153118
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.04290244414221921
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.033831658316686164
Metric: validation.head.optimistic.hits_at_1 = 0.5188624492164828
Metric: validation.tail.optimistic.hits_at_1 = 0.031050493325594893
Metric: validation.both.optimistic.hits_at_1 = 0.274956

In [15]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [16]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
            0
0   48.090027
1   10.891788
2    3.299072
3    2.153293
4    1.775846
..        ...
87   0.420241
88   0.417702
89   0.421031
90   0.418408
91   0.413737

[92 rows x 1 columns]


In [17]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.transductive_training.mapped_triples,
        additional_filter_triples=[
        dataset.transductive_training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/27.9k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.543506   0.009311     0.008661
hits_at_1                     0.534540   0.002944     0.002944
hits_at_3                     0.543408   0.005457     0.005457
hits_at_5                     0.544270   0.006319     0.006319
hits_at_10                    0.551702   0.017378     0.010233
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.025608   0.020538     0.019257
hits_at_1                     0.007360   0.005888     0.005888
hits_at_3                     0.019280   0.009550     0.008294
hits_at_5                     0.027287   0.020070     0.020070
hits_at_10                    0.044629   0.030195     0.029693
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.284557   0.014924     0.013959
hits_at_1                     0.270950   0.004416     0.004416
hits_at_3                     0.281344  

In [18]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.inductive_validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.inductive_validation.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/3.45k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.534042   0.024739     0.023362
hits_at_1                     0.518282   0.011608     0.011608
hits_at_3                     0.525247   0.014510     0.014510
hits_at_5                     0.530180   0.023796     0.019443
hits_at_10                    0.575160   0.034823     0.034823
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.056038   0.047740     0.045500
hits_at_1                     0.019153   0.017411     0.017411
hits_at_3                     0.063842   0.037725     0.036564
hits_at_5                     0.067905   0.046140     0.041788
hits_at_10                    0.108532   0.093442     0.092571
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.295040   0.036240     0.034431
hits_at_1                     0.268717   0.014510     0.014510
hits_at_3                     0.294544  

In [19]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.inductive_testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.inductive_validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/4.38k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.542285   0.023523     0.022376
hits_at_1                     0.528848   0.011631     0.011631
hits_at_3                     0.535006   0.014823     0.014823
hits_at_5                     0.539795   0.023033     0.019612
hits_at_10                    0.570125   0.031015     0.030787
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.056790   0.049593     0.047618
hits_at_1                     0.019156   0.018016     0.018016
hits_at_3                     0.064766   0.040821     0.040593
hits_at_5                     0.069555   0.048803     0.045154
hits_at_10                    0.111745   0.093957     0.093501
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.299538   0.036558     0.034997
hits_at_1                     0.274002   0.014823     0.014823
hits_at_3                     0.299886  

In [20]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [21]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [22]:
infodict['loss'] = NSSALoss
infodict['num_tokens'] = num_tokens
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience
infodict['aggregation'] = 'mlp'

In [23]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                      num_tokens   
17                   embedding_dim   
18                   learning_rate   
19                       optimizer   
20                      num_epochs   
21                        patience   
22                     aggregation   

                                                value  
0                              

In [24]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)